base model과 비교가 쉽도록, 어떤 부분을 지웠는지 별도로 표시했음. (더 복잡하려나요~ㅜ)

In [2]:
# ================================================================================
# 🔧 MECAB 토크나이저 사용 버전 (SentencePiece 제거)
# ================================================================================
# 원본 노트북과 동일한 코드 블록 구조로 작성
# ❌ 제거, ⚠️ 수정, ✅ 추가/유지

# ================================================================================
# ✅ Cell 0: Mecab 설치 (동일 - 유지)
# ================================================================================
!pip install konlpy
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab/
!bash install_mecab-ko_on_colab_light_220429.sh



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.9/495.9 kB 37.6 MB/s eta 0:00:00
Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 138 (delta 26), reused 22 (delta 8), pack-reused 91 (from 1)
Receiving objects: 100% (138/138), 1.72 MiB | 27.02 MiB/s, done.
Resolving deltas: 100% (65/65), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2025-11-13 12:05:14--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.142.25, 104.192.142.24, 104.192.142.26, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.142.25|:443... connected.
HTTP request sent,

코랩에 수동으로 데이터 업로딩 !!!

rating_test.txt와 rating_train.txt를 코랩에 복사해주세요. 이거 잊으면 안돼요. 한 코드내에서 깃업에서 중복 클론해도 문제가 없는지, 해결책을 찾지 못해서 ... 일단 수동으로 데이터를 넣었어요.

In [3]:
# ================================================================================
# ✅ Cell 1: 라이브러리 임포트 (동일 - 유지)
# ================================================================================
# 라이브러리 싹 다 모아봤습니다.
import torch
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import konlpy

print(torch.__version__)
print(np.__version__)
print(pd.__version__)
print(matplotlib.__version__)
print(konlpy.__version__)

# ✅ Mecab 로드 (추가)
from konlpy.tag import Mecab
from collections import Counter
from tqdm import tqdm

mecab = Mecab()
print("\n✅ Mecab 토크나이저 로드 완료")


# ================================================================================
# ✅ Cell 2: 데이터 로드 (동일 - 유지)
# ================================================================================
train = pd.read_table("/content/ratings_train.txt")
test = pd.read_table("/content/ratings_test.txt")

print(train.info())
train.head()


# ================================================================================
# ✅ Cell 3: 전처리 (동일 - 유지)
# ================================================================================
# 팀 전처리 코드
import re

# 불용어 리스트
STOPWORDS = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# 맞춤법 변형 사전
SPELLING_DICT = {
    '굳': ['궅', '굳', '굿'],
    '미쳤': ['미첫', '미쳣', '미첬', '미쳤', 'ㅁㅊ'],
    '괜찮': ['괜찮', '괜춘', '괜찬', 'ㄱㅊ', '갠찬', '갠찮', '괸찬', '괸찮'],
    '봤': ['봣'],
    '겠': ['겟']
}

# 텍스트 컬럼명
TEXT_COL = "document"

def preprocess_text(text):
    """
    텍스트 전처리 함수
    """
    if pd.isna(text):
        return ""

    text = str(text)

    # 1. 반복되는 문장 부호 제거 (2개 이상 → 1개)
    text = re.sub(r'([.!?…;])\1+', r'\1', text)

    # 2. 맞춤법 변형 통일
    for correct, variations in SPELLING_DICT.items():
        for variant in variations:
            text = text.replace(variant, correct)

    # 3. 반복 문자 제거 (3번 이상 반복 → 2번)
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)

    # 4. 자음/모음 단독 제거 (완성형 한글 필터링 전에 먼저 제거)
    # 한글 자음: ㄱ-ㅎ, 한글 모음: ㅏ-ㅣ
    text = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+', ' ', text)

    # 5. 영어/숫자/한글/문장부호만 남기고 모두 삭제
    # 이모지, 이모티콘, 특수문자 자동 제거
    text = re.sub(r'[^가-힣a-zA-Z0-9\s.!?,]', ' ', text)

    # 6. 문장부호 앞뒤로 공백 추가
    text = re.sub(r'([.!?,])', r' \1 ', text)

    # 7. 영어 소문자 변환
    text = text.lower()

    # 9. 불용어 제거 (조사가 붙은 경우도 처리)
    words = text.split()

    # 단어 끝에 불용어가 붙어있으면 제거
    filtered_words = []
    for word in words:
        # 단어 전체가 불용어인 경우
        if word in STOPWORDS:
            continue
        # 단어 끝에서 불용어 제거 (가장 긴 것부터 체크)
        # 예: "학교에서" -> "학교", "영화는" -> "영화"
        found = False
        for stopword in sorted(STOPWORDS, key=len, reverse=True):
            if len(word) > len(stopword) and word.endswith(stopword):
                cleaned = word[:-len(stopword)]
                # 남은 부분이 2글자 이상일 때만 제거
                # 굳이 -> 굳, 같이 -> 같  이렇게 바뀌어서 길이 제한 추가했습니다.
                # "굳이" -> "굳" (X), "영화는" -> "영화" (O)
                if len(cleaned) >= 2:
                    filtered_words.append(cleaned)
                    found = True
                    break
        if not found:
            filtered_words.append(word)

    text = ' '.join(filtered_words)

    # 10. 연속 공백을 하나의 공백으로 교체
    text = re.sub(r'\s+', ' ', text)

    # 11. 앞뒤 공백 제거
    text = text.strip()

    return text


def preprocess_dataframe(df, text_col=TEXT_COL):
    """
    데이터프레임 전처리 함수
    """
    print(f"전처리 전 데이터 크기: {len(df)}")

    # 1. 결측치 제거
    df = df.dropna(subset=[text_col])
    print(f"결측치 제거 후: {len(df)}")

    # 2. 텍스트 전처리 적용
    df[text_col] = df[text_col].apply(preprocess_text)

    # 3. 전처리 후 빈 문자열 제거
    df = df[df[text_col].str.strip() != '']
    print(f"빈 문자열 제거 후: {len(df)}")

    # 4. 중복 행 제거
    df = df.drop_duplicates(subset=[text_col])
    print(f"중복 제거 후: {len(df)}")

    # 인덱스 재설정
    df = df.reset_index(drop=True)

    return df


# 샘플 텍스트로 테스트
sample_texts = [
    "어제 본 영화 진짜 재밌었음!!! 또 보고 싶어 😂",
    "나는 오늘 아침에 학교에 갔다. 근데 너무 졸렸음ㅋㅋㅋㅋ",
    "밥은 먹었니?? 아직이야... 점심에 같이 먹자!!!",
    "메캅 형태소 분석은 한국어 처리에서 많이 사용돼 👍",
    "파이썬으로 토큰 빈도와 품사 분포를 시각화해 보자!!!",
    "요즘 코사인 유사도 기반 벡터 검색으로 RG 구축을 많이 해!!",
    "에이전트는 외부 도구를 호출해 작업을 자동화할 수 있어. 굳!",
    "이 영화 진짜 미쳣다!!! 너무 재밌음ㅋㅋㅋㅋ",
    "배우 연기 굳이 훌륭했음, 스토리는 봣지만...",
    "이건 ㄱㅊ 영화네, 굿굿!"
]

print("=" * 80)
print("샘플 텍스트 전처리 결과")
print("=" * 80)
for i, text in enumerate(sample_texts, 1):
    processed = preprocess_text(text)
    print(f"\n[{i}] 원본: {text}")
    print(f"    결과: {processed}")

print("\n" + "=" * 80)
print("실제 데이터 적용 예시")
print("=" * 80)

train_processed = preprocess_dataframe(train.copy())
test_processed = preprocess_dataframe(test.copy())

print("\n전처리 완료!")
print(f"Train 데이터: {len(train_processed)}개")
print(f"Test 데이터: {len(test_processed)}개")

print("\n전처리 결과 샘플:")
print(train_processed.head(10))


# ================================================================================
# ❌ Cell 4: SentencePiece 설치 [제거]
# ================================================================================
# ❌ 제거됨: !pip install sentencepiece


# ================================================================================
# ❌ Cell 5: SentencePiece 학습 [제거]
# ================================================================================
# ❌ 제거됨: SentencePiece 학습 코드


# ================================================================================
# ❌ Cell 6: SentencePiece 모델 활용 예시 [제거]
# ================================================================================
# ❌ 제거됨: SentencePiece 테스트 코드


# ================================================================================
# ⚠️ Cell 7: 토크나이징 함수 [수정] - Vocab Size 8000 고정
# ================================================================================

# ⚠️ 수정: Vocab Size 8000으로 제한
def mecab_tokenize(mecab, corpus, max_vocab_size=8000):
    """
    Mecab 형태소 분석기로 토크나이제이션

    ⚠️ 수정: vocab_size를 8000으로 고정하여 메모리 최적화

    Args:
        mecab: Mecab 토크나이저 객체
        corpus: 전처리된 텍스트 리스트
        max_vocab_size: 최대 vocab 크기 (8000 고정)

    Returns:
        tensor: 패딩된 토큰 ID 텐서
        word_index: 단어 -> 인덱스 (상위 8000개)
        index_word: 인덱스 -> 단어
    """
    # Step 1: Mecab 형태소 분석
    all_tokens = []
    tokenized_texts = []

    print("🔄 Mecab 형태소 분석 진행 중...")
    for text in tqdm(corpus, desc="Mecab tokenizing"):
        tokens = mecab.morphs(text)  # 형태소 리스트
        tokenized_texts.append(tokens)
        all_tokens.extend(tokens)

    # Step 2: 단어 빈도 계산
    token_counter = Counter(all_tokens)

    print(f"\n📊 전체 고유 형태소 수: {len(token_counter):,}")
    print(f"   상위 10개: {token_counter.most_common(10)}\n")

    # ⚠️ 수정: 상위 8000개 형태소만 선택
    print(f"⚡ Vocab Size를 {max_vocab_size:,}로 제한 중...")
    top_tokens = dict(token_counter.most_common(max_vocab_size - 1))  # <PAD> 예약

    # Step 3: 단어 인덱스 생성 (상위 8000개만)
    word_index = {}
    word_index['<PAD>'] = 0  # 인덱스 0: 패딩

    for idx, (word, count) in enumerate(top_tokens.items(), start=1):
        word_index[word] = idx

    # Step 4: 역 매핑
    index_word = {idx: word for word, idx in word_index.items()}

    print(f"✅ 최종 Vocab 크기: {len(word_index):,}\n")

    # Step 5: ID 변환 (미지의 토큰은 0(PAD)으로 처리)
    tensor_list = []
    unknown_count = 0
    total_tokens = 0

    for tokens in tqdm(tokenized_texts, desc="Converting to tensor"):
        token_ids = []
        for token in tokens:
            if token in word_index:
                token_ids.append(word_index[token])
            else:
                # ⚠️ 미지의 토큰은 0(PAD)으로 처리
                token_ids.append(0)
                unknown_count += 1
            total_tokens += 1

        tensor_list.append(torch.tensor(token_ids, dtype=torch.long))

    # Step 6: 패딩
    tensor = pad_sequence(tensor_list, batch_first=True, padding_value=0)

    print(f"\n✅ 토크나이제이션 완료!")
    print(f"   최종 텐서 shape: {tensor.shape}")
    print(f"   OOV (미지 토큰) 비율: {unknown_count/total_tokens*100:.2f}% ({unknown_count:,}/{total_tokens:,})\n")

    return tensor, word_index, index_word


# ================================================================================
# ⚠️ Cell 8: DataLoader 생성 함수 [수정]
# ================================================================================
from torch.utils.data import TensorDataset, DataLoader, random_split

# ⚠️ 수정: attention_masks 제거
def create_dataloaders(train_input_ids, train_labels,
                       test_input_ids, test_labels,
                       batch_size=32, val_ratio=0.2):
    """
    Train/Val/Test DataLoader 생성 (Mecab 버전)

    Args:
        train_input_ids: 훈련 데이터 입력 ID 텐서
        train_labels: 훈련 레이블
        test_input_ids: 테스트 데이터 입력 ID 텐서
        test_labels: 테스트 레이블
        batch_size: 배치 크기
        val_ratio: validation 비율

    Returns:
        train_loader, val_loader, test_loader
    """
    # Train 데이터에서 train/val split
    train_dataset = TensorDataset(train_input_ids, train_labels)  # ✅ attention_masks 제거
    val_size = int(len(train_dataset) * val_ratio)
    train_size = len(train_dataset) - val_size
    train_split, val_split = random_split(train_dataset, [train_size, val_size])

    # DataLoader 생성
    train_loader = DataLoader(train_split, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_split, batch_size=batch_size, shuffle=False)

    # Test DataLoader
    test_dataset = TensorDataset(test_input_ids, test_labels)  # ✅ attention_masks 제거
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    print(f"✅ DataLoader 생성 완료!")
    print(f"   Train samples: {train_size:,} → {len(train_loader)} batches")
    print(f"   Val samples: {val_size:,} → {len(val_loader)} batches")
    print(f"   Test samples: {len(test_dataset):,} → {len(test_loader)} batches")
    print(f"   Batch size: {batch_size}")

    return train_loader, val_loader, test_loader


# ✅ 추가: 길이 제한 함수
def truncate_corpus(corpus, max_length=130):
    """텍스트 길이 제한"""
    truncated_corpus = []
    for text in tqdm(corpus, desc="Truncating corpus"):
        if len(text) > max_length * 5:
            truncated_corpus.append(text[:max_length * 5])
        else:
            truncated_corpus.append(text)
    return truncated_corpus


# 데이터 준비
corpus = train_processed['document'].tolist()
labels = torch.tensor(train_processed['label'].values, dtype=torch.long)

print(f"📊 원본 데이터:")
print(f"   Corpus 크기: {len(corpus):,}")
print(f"   Labels 크기: {len(labels):,}\n")

corpus_truncated = truncate_corpus(corpus, 130)
print()

# ✅ Mecab 토크나이징 (vocab_size=8000 고정)
train_input_ids, train_word_index, train_index_word = mecab_tokenize(mecab, corpus_truncated, max_vocab_size=8000)
print()

# Test 데이터도 처리
test_corpus = test_processed['document'].tolist()
test_labels = torch.tensor(test_processed['label'].values, dtype=torch.long)

test_corpus_truncated = truncate_corpus(test_corpus, 130)
test_input_ids, _, _ = mecab_tokenize(mecab, test_corpus_truncated, max_vocab_size=8000)
print()

# DataLoader 생성
train_loader, val_loader, test_loader = create_dataloaders(
    train_input_ids,
    labels,
    test_input_ids,
    test_labels,
    batch_size=32,
    val_ratio=0.2
)

# 확인
print("=" * 60)
print("📊 DataLoader 샘플 확인")
print("=" * 60)

# ✅ 2개만 언팩
for batch in train_loader:
    x, y = batch  # ✅ 2개만 언팩
    print(f"Train - 입력 텐서 shape: {x.shape}")
    print(f"Train - 레이블 shape: {y.shape}")
    break

for batch in val_loader:
    x, y = batch  # ✅ 2개만 언팩
    print(f"Val - 입력 텐서 shape: {x.shape}")
    print(f"Val - 레이블 shape: {y.shape}")
    break

for batch in test_loader:
    x, y = batch  # ✅ 2개만 언팩
    print(f"Test - 입력 텐서 shape: {x.shape}")
    print(f"Test - 레이블 shape: {y.shape}")
    break


# ================================================================================
# ✅ Cell 9: (빈 셀 유지)
# ================================================================================


# ================================================================================
# ✅ Cell 10: 하이퍼파라미터 정의 [수정]
# ================================================================================
import torch.nn as nn
import torch.nn.functional as F

# 공통 하이퍼파라미터 정의
# ✅ 수정: VOCAB_SIZE 8000으로 고정 (동적화 안 함)
VOCAB_SIZE = 8000  # ✅ 8000 고정 (메모리 최적화)
EMBEDDING_DIM = 100                # 임베딩 벡터의 차원
HIDDEN_DIM = 128                   # LSTM의 은닉 상태 차원
OUTPUT_DIM = 1                     # 출력 차원 (긍정=1, 부정=0 -> 1개)
N_LAYERS = 2                       # LSTM 레이어 개수
BIDIRECTIONAL = True               # 양방향 RNN/LSTM 여부
DROPOUT_RATE = 0.2                 # 드롭아웃 비율
PAD_IDX = 0                        # 패딩 인덱스 (0)

print(f"📊 모델 하이퍼파라미터:")
print(f"   VOCAB_SIZE: {VOCAB_SIZE:,} (고정)")


# ================================================================================
# ✅ Cell 11: LSTM 모델 [동일 - 유지]
# ================================================================================
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers,
                 bidirectional, dropout, pad_idx):
        super().__init__()

        # 1. 임베딩 레이어
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)

        # 2. LSTM 레이어
        self.lstm = nn.LSTM(input_size=embedding_dim,
                           hidden_size=hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional,
                           batch_first=True,
                           dropout=dropout)

        # 3. FC 레이어
        fc_input_dim = hidden_dim * 2
        self.fc = nn.Linear(fc_input_dim, output_dim)

        # 4. 드롭아웃
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        embedded = self.embedding(text)

        # 2. LSTM
        _output, (hidden, cell) = self.lstm(embedded)

        # 3. 마지막 레이어의 은닉 상태 결합 (양방향 처리)
        if self.lstm.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        else:
            hidden = self.dropout(hidden[-1,:,:])

        # 4. FC 레이어 통과
        prediction = self.fc(hidden)

        return prediction.squeeze(1)


# ================================================================================
# ✅ Cell 12: 훈련 함수 [수정]
# ================================================================================
import torch
import torch.optim as optim
import time

# 0. GPU 장치 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 1. 헬퍼 함수 정의
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

# 2. 훈련 함수 정의
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    # ✅ 수정: attention_masks 제거
    for texts, labels in iterator:  # ✅ 2개만 언팩
        texts = texts.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        predictions = model(texts)
        loss = criterion(predictions, labels.float())
        acc = binary_accuracy(predictions, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

# 3. 평가 함수 정의
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        # ✅ 수정: attention_masks 제거
        for texts, labels in iterator:  # ✅ 2개만 언팩
            texts = texts.to(device)
            labels = labels.to(device)
            predictions = model(texts)
            loss = criterion(predictions, labels.float())
            acc = binary_accuracy(predictions, labels)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


# ================================================================================
# ✅ Cell 13: 모델 훈련 [동일 - 유지]
# ================================================================================
# 모델 설정
lstm_model = LSTMModel(
    VOCAB_SIZE,
    EMBEDDING_DIM,
    HIDDEN_DIM,
    OUTPUT_DIM,
    N_LAYERS,
    BIDIRECTIONAL,
    DROPOUT_RATE,
    PAD_IDX
).to(device)

save_path = 'best_model_lstm.pt'
N_EPOCHS = 20
patience = 5
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.Adam(filter(lambda p: p.requires_grad, lstm_model.parameters()), lr=0.0001)

# Early stopping 변수
best_valid_loss = float('inf')
patience_counter = 0

train_losses, train_accs, valid_losses, valid_accs = [], [], [], []

print(f"\n{'='*60}")
print(f"--- LSTM Model Training starts ---")
print(f"{'='*60}\n")

# 학습 루프
for epoch in range(N_EPOCHS):
    start_time = time.time()

    train_loss, train_acc = train(lstm_model, train_loader, optimizer, criterion)
    valid_loss, valid_acc = evaluate(lstm_model, val_loader, criterion)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

    # 기록
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    valid_losses.append(valid_loss)
    valid_accs.append(valid_acc)

    # Early Stopping
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(lstm_model.state_dict(), save_path)
        patience_counter = 0
        print(f'\t>> Validation loss improved ({best_valid_loss:.3f}). Model saved.')
    else:
        patience_counter += 1
        print(f'\t>> Validation loss did not improve. Counter: {patience_counter}/{patience}')
        if patience_counter >= patience:
            print(f'--- Early stopping triggered after {epoch+1} epochs ---')
            break

# 테스트 평가
print(f"\n--- Loading best LSTM model for test evaluation ---")
lstm_model.load_state_dict(torch.load(save_path))
test_loss, test_acc = evaluate(lstm_model, test_loader, criterion)

print(f"\n--- LSTM Model Test Results (Best Model) ---")
print(f'\tTest Loss: {test_loss:.3f}')
print(f'\tTest Acc:  {test_acc*100:.2f}%')

# ================================================================================
# 🎉 코드 끝
# ================================================================================

2.8.0+cu126
2.0.2
2.2.2
3.10.0
0.6.0

✅ Mecab 토크나이저 로드 완료
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB
None
샘플 텍스트 전처리 결과

[1] 원본: 어제 본 영화 진짜 재밌었음!!! 또 보고 싶어 😂
    결과: 어제 본 영화 진짜 재밌었음 ! 또 보고 싶어

[2] 원본: 나는 오늘 아침에 학교에 갔다. 근데 너무 졸렸음ㅋㅋㅋㅋ
    결과: 나는 오늘 아침 학교 갔다 . 근데 너무 졸렸음

[3] 원본: 밥은 먹었니?? 아직이야... 점심에 같이 먹자!!!
    결과: 밥은 먹었니 ? 아직이야 . 점심 같이 먹자 !

[4] 원본: 메캅 형태소 분석은 한국어 처리에서 많이 사용돼 👍
    결과: 메캅 형태소 분석 한국어 처리에서 많이 사용돼

[5] 원본: 파이썬으로 토큰 빈도와 품사 분포를 시각화해 보자!!!
    결과: 파이썬 토큰 빈도 품사 분포 시각화해 보자 !

[6] 원본: 요즘 코사인 유사도 기반 벡터 검색으로 RG 구축을 많이 해!!
    결과: 요즘 코사인 유사 기반 벡터 검색 rg 구축을 많이 해 !

[7] 원본: 에이전트는 외부 도구를 호출해 작업을 자동화할 수 있어. 굳!
    결과: 에이전트 외부 도구 호출해 작업을 자동화할 수 있어 . 굳 !

[8] 원본: 이 영화 진짜 미쳣다!!! 너무 

Truncating corpus: 100%|██████████| 144478/144478 [00:00<00:00, 3990942.13it/s]



🔄 Mecab 형태소 분석 진행 중...


Mecab tokenizing: 100%|██████████| 144478/144478 [00:08<00:00, 17929.09it/s]



📊 전체 고유 형태소 수: 48,645
   상위 10개: [('.', 130475), ('영화', 57208), ('다', 53641), ('고', 47096), ('하', 40099), ('이', 35465), ('을', 29883), ('는', 26938), ('보', 24138), ('게', 22199)]

⚡ Vocab Size를 8,000로 제한 중...
✅ 최종 Vocab 크기: 8,000



...

🔄 Mecab 형태소 분석 진행 중...


Mecab tokenizing: 100%|██████████| 48700/48700 [00:02<00:00, 19687.76it/s]



📊 전체 고유 형태소 수: 29,887
   상위 10개: [('.', 43474), ('영화', 19538), ('다', 18184), ('고', 15643), ('하', 13439), ('이', 11872), ('을', 9922), ('는', 9100), ('보', 8073), ('게', 7613)]

⚡ Vocab Size를 8,000로 제한 중...
✅ 최종 Vocab 크기: 8,000



Converting to tensor: 100%|██████████| 48700/48700 [00:00<00:00, 97268.97it/s]



✅ 토크나이제이션 완료!
   최종 텐서 shape: torch.Size([48700, 105])
   OOV (미지 토큰) 비율: 4.48% (35,264/787,110)


✅ DataLoader 생성 완료!
   Train samples: 115,583 → 3612 batches
   Val samples: 28,895 → 903 batches
   Test samples: 48,700 → 1522 batches
   Batch size: 32
📊 DataLoader 샘플 확인
Train - 입력 텐서 shape: torch.Size([32, 96])
Train - 레이블 shape: torch.Size([32])
Val - 입력 텐서 shape: torch.Size([32, 96])
Val - 레이블 shape: torch.Size([32])
Test - 입력 텐서 shape: torch.Size([32, 105])
Test - 레이블 shape: torch.Size([32])
📊 모델 하이퍼파라미터:
   VOCAB_SIZE: 8,000 (고정)
Using device: cuda

--- LSTM Model Training starts ---

Epoch: 01 | Time: 0m 28s
	Train Loss: 0.501 | Train Acc: 74.74%
	 Val. Loss: 0.428 |  Val. Acc: 80.09%
	>> Validation loss improved (0.428). Model saved.
Epoch: 02 | Time: 0m 27s
	Train Loss: 0.400 | Train Acc: 81.89%
	 Val. Loss: 0.392 |  Val. Acc: 82.54%
	>> Validation loss improved (0.392). Model saved.
Epoch: 03 | Time: 0m 28s
	Train Loss: 0.366 | Train Acc: 83.85%
	 Val. Loss: 0.375 |  Val. Ac